In [1]:
import os
import json
from qiskit import IBMQ

this_dir = os.getcwd()

program_path = os.path.join(this_dir, 'schwinger_rqd.py')

In [2]:
provider = IBMQ.load_account()

## Write the program file

In [3]:
output = open(program_path, 'w')
main_source = open(os.path.join(this_dir, 'main.py'))

while True:
    line = main_source.readline()
    if not line:
        break
        
    if line.startswith('## include'):
        nextline = main_source.readline()
        if nextline.startswith('from ') or nextline.startswith('import '):
            module = nextline.split()[1]
        else:
            raise RuntimeError('Invalid include block: {}'.format(nextline.strip()))
            
        include_path = os.path.join(line.split()[2], *module.split('.')) + '.py'
        
        with open(include_path) as source:
            output.write(source.read())
            output.write('\n')
            
    elif line.startswith('## ignore'):
        main_source.readline()
    else:
        output.write(line)
    
output.close()
main_source.close()

## Set the metadata dict

In [4]:
json_schema = 'https://json-schema.org/draft/2019-09/schema'

metadata = {
    'name': 'schwinger-rqd',
    'description': 'Restarted Quantum Dynamics simulation of the lattice Schwinger model.',
    'max_execution_time': 14400,
    'spec': {
        'backend_requirements': {'min_num_qubits': 2},
        'parameters': {
            '$schema': json_schema,
            'type': 'object',
            'properties': {
                'num_sites': {'description': 'Number of lattice sites (qubits).', 'type': 'number'},
                'aJ': {'description': 'Square of the product of the coupling constant and the lattice spacing, divided by 2.', 'type': 'number'},
                'am': {'description': 'Mass times the lattice spacing.', 'type': 'number'},
                'omegadt': {'description': 'Trotter time step divided by double the lattice spacing.', 'type': 'number'},
                'num_tsteps': {'description': 'Number of Trotter steps to simulate.', 'type': 'number'},
                'tsteps_per_rqd': {'description': 'Number of Trotter steps to advance in each RQD step.', 'type': 'number'},
                'api_token': {'description': 'IBM Quantum API token.', 'type': 'string'},
                'physical_qubits': {'description': 'List of physical qubits to use.', 'type': 'array'},
                'error_matrix': {'description': 'Measurement error matrix to be passed to the MeasurementFilter.', 'type': 'array'},
                'max_sweeps': {'description': 'Number of parameter sweeps for sequential minimum optimization.', 'type': 'number'},
                'minimizer_shots_per_job': {'description': 'Number of shots per job to use for sequential minimum optimization.', 'type': 'number'},
                'minimizer_jobs': {'description': 'Number of jobs to use for sequential minimum optimization.', 'type': 'number'},
                'forward_shots': {'description': 'Number of shots to use for the forward steps of RQD.', 'type': 'number'},
                'resume_from': {'description': 'Minimizer parameters to resume RQD.', 'type': 'object'}
            },
            'required': ['num_sites', 'aJ', 'am', 'omegadt', 'num_tsteps', 'tsteps_per_rqd']
        },
        'return_values': {'$schema': json_schema, 'description': 'All done!', 'type': 'string'},
        'interim_results': {
            '$schema': json_schema,
            'type': 'object',
            'properties': {
                'rqd_step': {'description': 'RQD iteration number.', 'type': 'number'},
                'forward_counts': {'description': 'List of counts dictionaries of the Trotter steps in one RQD step.', 'type': 'array'},
                'optimal_params': {'description': 'Optimal ansatz parameter values from the previous RQD step.', 'type': 'array'},
                'shots_values': {'description': 'Array of cumulative shot numbers during the RQD iteration', 'type': 'array'},
                'cost_values': {'description': 'Array of the cost value evolution during the RQD iteration', 'type': 'array'},
                'state': {'description': 'Current state of the minimizer.', 'type': 'object'},
                'minimizer_state': {'description': 'Current strategy-specific state of the minimizer.', 'type': 'object'}
            }
        }
    }
}

## Upload the program

In [5]:
program_id = 'schwinger-rqd-oGExmQ8LjP'

if not program_id:
    program_id = provider.runtime.upload_program(
        data=program_path,
        metadata=metadata)
    
    print(program_id)

else:
    provider.runtime.update_program(
        program_id,
        data=program_path,
        metadata=metadata)